# Tickable 10 - model solution

*&#169; Eike Mueller, University of Bath 2019-2021. This model solution is copyright of Eike Mueller, University of Bath. It is provided exclusively for educational purposes at the University and is to be downloaded or copied for your private study only. Further distribution, e.g. by upload to external repositories, is prohibited.*

### &#9745; Task 1
The implementation of the `mul()` and `add()` functions can be found in the file `polynomial.py`.

This file contains two additional functions which make handling of polynomials a bit easier: The function `_simplify()` ensure that the highest order term in polynomials with positive degree is non-zero. This is achieved by recursively chopping off any terms of the from `[0,1]` from the end of the list that stores the polynomial. For example

```Python
_simplify([[1,3],[-4,7],[0,1],[0,1]])
```

will return `[[1,3],[-4,7]]`.

The function `to_str()` converts a polynomial to a string that can be printed out. For example, for the polynomial

$$
\frac{2}{9}+\frac{1}{4}x+\frac{2}{3}x^2
$$

```Python
print(polynomial.to_str([[2,9],[1,4],[2,3]]))
```

will print out

```Python
+ (2 / 9) * x^0 + (1 / 4) * x^1 + (2 / 3) * x^2
```

Note that all functions are documented with a docstring, and there is a docstring at the head of the module with general information.

### &#9745; Task 2
See the tests in `test_polynomial.py`.

### &#9745; Task 3
#### Task 3a
To import the module and run the tests use:

In [ ]:
import polynomial
!pytest -v test_polynomial.py

The `help` command will produce additional information on the module and list the functions it contains. Note that the `_simplify()` function is not listed since it starts with an underscore (`_`).

In [ ]:
help(polynomial)

#### Task 3b
To compute the $n$-th Legendre polynomial we can use the following recursive function:

In [ ]:
def legendre_poly(n):
    '''Compute n-th Legendre polynomial P_n(x)
    
    :arg n: Polynomial degree n
    '''
    if n==0:
        return [[1,1],]
    elif n==1:
        return [[0,1],[1,1],]
    else:
        q_1 = [[0,1],[2*n-1,n]] # polynomial (2n-1)/n*x that multiplies P_{n-1}(x)
        q_0 = [[-(n-1),n],]     # polynomial -(n-1)/n that multiplies P_{n-2}(x)
        return polynomial.add( polynomial.mul(q_1,legendre_poly(n-1)), polynomial.mul(q_0,legendre_poly(n-2)) )

(as for the Fibonacci numbers, this recursive implementation becomes inefficient for large values of $n$). Note that by writing `polynomial.add()` etc. we ensure that we use the `mul()` function for polynomials, which is different from the `mul()` function for rational numbers defined in `rational.py`.

We find in particular:

In [ ]:
print(polynomial.to_str(legendre_poly(10)))

This implies that

$$
P_{10}(x) = -\frac{63}{256} + \frac{3465}{256}x^2 - \frac{15015}{128}x^4+\frac{45045}{128}x^6 - \frac{109395}{256}x^8 + \frac{46189}{256}x^{10}
$$

### Other things to try out
Functions for evaluating and integrating polynomials are implemented in `polynomial.py`, which also contains tests for these functions.

To check the orthogonality relation we can proceed as follows for given degrees $n$, $m$:

* Compute $P_n(x)$ and $P_m(x)$ with `legendre_polynomial()`
* Multiply these two polynomials with the `mul()` function to obtain a polynomial $q_{n,m}(x)$
* Use the `integrate()` function to integrate $q_{n,m}(x)$. This results in a polynomial $\tilde{Q}_{n,m}(x)$.
* Use the `mul()` function to multiply this polynomial $\tilde{Q}_{n,m}(x)$ by the zero-order polynomial $\frac{2n+1}{2}$ to obtain $Q_{n,m}(x)$.
* Evaluate $Q_{n,m}(x)$ at $x=1$ and $x=-1$ with the `evaluate()` function to obtain the final result
$$
\Omega_{n,m} = Q_{n,m}(1) - Q_{n,m}(-1)=\frac{2n+1}{2}\int_{-1}^{1} P_n(x) P_m(x)\;dx
$$

The following code computes $Q_{n,m}$ and prints out the results in a table.

In [ ]:
import rational

for n in range(16):
    for m in range(16):
        P_n = legendre_poly(n)
        P_m = legendre_poly(m)
        q_nm = polynomial.mul(P_n,P_m)
        tilde_Q_nm = polynomial.integrate(q_nm)
        Q_nm = polynomial.mul([[2*n+1,2],],tilde_Q_nm)
        Q_nm_p1 = polynomial.evaluate(Q_nm,[+1,1])
        Q_nm_m1 = polynomial.evaluate(Q_nm,[-1,1])
        r = rational.to_str(rational.add(Q_nm_p1,rational.mul([-1,1],Q_nm_m1)))
        print (r," ",end="")
    print ()